# Formation RNAseq CEA - juin 2023

*Enseignantes : Sandrine Caburet et Claire Vandiedonck*

Session IFB : 5 CPU + 21 GB de RAM

# Part 9: DESeq2 Normalisation and Differential Expression analysis


- 0. 1 - Setting up this R session on IFB core cluster  
- 0. 2 - Parameters to be set or modified by the user   
- 1 - Loading input data and metadata   
- 2 - Normalisation with 'DESeq2   
- 3 - Differential expression analysis with `DESeq2`   
- 4 - Saving our results for later use : RData file

---

## 0.1 - Setting up this R session on IFB core cluster

<em>loaded JupyterLab</em> : Version 3.2.1

In [ ]:
## Code cell 1 ##

session_parameters <- function(){
    
    jupytersession <- c(system('echo "=== Cell launched on $(date) ==="', intern = TRUE),
                        system('squeue -hu $USER', intern = TRUE))
    
    jobid <- system("squeue -hu $USER | awk '/jupyter/ {print $1}'", intern = TRUE)
    jupytersession <- c(jupytersession,
                        "=== Current IFB session size: Medium (5CPU, 21 GB) ===",
                        system(paste("sacct --format=JobID,AllocCPUS,NODELIST -j", jobid), intern = TRUE))
    print(jupytersession[1:6])
    
    return(invisible(NULL))
}

session_parameters()

__

Next we load into this R session the various tools that we will use.   
***DO NOT worry*** if you see a large red output!!   
You should see this large red output only once, when the relevant packages are installed in your home directory. Afterwards, they will be detected as present, and this large red output won't show if you run the cell another time.

In [ ]:
## Code cell 2 ##

if (!require("DESeq2", quietly = TRUE)) {
    install.packages("DESeq2")}

library("matrixStats")
library("DESeq2", quietly = TRUE)

In [ ]:
## Code cell 3 ##   

cat("Voici mon environnement de travail avec les paquets de R chargés:\n")
sessionInfo()

---

## 0.2 - Parameters to be set or modified by the user


- Using a full path with a `/` at the end, **define the folder** of the project as  `gohome` variable, and the folder where you work as the `myfolder` variable:

In [ ]:
## Code cell 4 ##


gohome <- "/shared/projects/2312_rnaseq_cea/"
gohome

# In a Jupyter Hub and a jupyter notebook in R, by default the working directory is where the notebook is opened
getwd()
myfolder <- getwd()
myfolder


- With a `/` at the end, define the path to the folder where the results of this normalisation by `DESeq2` will be stored:

In [ ]:
## Code cell 5 ##

# creation of the directory, recursive = TRUE is equivalent to the mkdir -p in Unix
dir.create(paste(myfolder,"/Results/deseq2/", sep = ""), recursive = TRUE)

# storing the path to this output folder in a variable
deseq2folder <- paste(myfolder,"/Results/deseq2/", sep = "")
deseq2folder

# listing the content of the folder
print(system(paste("ls -hlt", deseq2folder), intern = TRUE) )

- Last, we specify the size of the graphical outputs that will be used for all the plots in the notebook.    
This setting could be modified at will for each plot. 

In [ ]:
## Code cell 6 ##

options(repr.plot.width=15, repr.plot.height=8)

## 1 - Loading input data and metadata

We now need to retrieve the counts data and the metadata about samples.   
As we stored those in a global Rdata objet at the end of Pipe_08, we can simply reload all our information by openig this Rdata object.  


- the read counts produced by featureCounts (as in the Pipe_6 notebook), on all 11 samples   
- a metadata file, providing information about the samples, in particular the conditions of the experiment. 

In [ ]:
## Code cell 7 ##

pca1folder <- paste0(myfolder,"/Results/pca1/")
pca1folder
rdata <- paste0(pca1folder,"AllMyData.RData")
rdata
load(rdata,verbose = T)

We can now list all the object we have currently in our session: 

In [ ]:
## Code cell 8 ##

ls() 

- `countdata`is a dataframe containing all the read counts produced by `featureCounts` (as in the Pipe_06 notebook), on all 11 samples, including low counts genes.
- `countdata2`is the same dataframe without the low counts genes      
- `samples`is a dataframe containing the information about the samples, in particular the conditions of the experiment. 

We don't need anything else from the previous session, so we can remove the other objects from our current R session, in order to save memory.

In [ ]:
## Code cell 9 ##

rm(logcounts, counts, clusters, keep, pca1folder, PCAdata, tlogcounts)  
ls()

We verify that the counts dataframes look like what we expect:

In [ ]:
## Code cell 10 ##

head(countdata)
head(countdata2)

In [ ]:
## Code cell 11 ##

head(samples, n=13)

To ensure the metadata `samples` and `countdata` dataframes have the same samples names, we run the code below.    
This will return TRUE if the names in both files are the same. In all there are 11 samples in both files.

In [ ]:
## Code cell 12 ##

table(colnames(countdata)==samples$SampleID)

**Great!!**. All is set now to perform normalisation and differential expression analysis with `DESeq2`.

---
## 2 - Normalisation with 'DESeq2'

All the steps required to perform normalisation of read counts and differential expression analysis has been placed into a single function, DESeq. More details on the respective steps can be found in the DESeq2 paper. You could also type and run ?DESeq to read more on the steps.

### 2.1 - Create the DESeqDataSet object from the countdata and metadata

DESeq2 uses an object called DESeqDataSet to store read counts and other intermediate calculations needed for normalisation and differential expression analysis. This is represented in the code below as dds (d for DESeq, d for Data and s for set). The basic idea behind this is to coerce the countdata and metadata into a single format that DESeq2 can accept and work with. We do this with the DESeqDataSetFromMatrix function.

In [ ]:
## Code cell 13 ##

nrow(countdata)
nrow(countdata2)

# 57010 in countdata
# 22927 in countdata2

There are 57010 genes in our samples, because the genes with a null or a low expression were not filtered out.   
`DESeq2` takes care of these genes along the way, so we can keep those.   
Nevertheless, if you work in a low-memory setting, it is preferable to reduce the size of the matrix to ease its handling, by removing the low-expression genes the same way we did in Pipe_08, of by using as input the `countData2`dataframe that is already computed.   
Here, we will go on with the `countdata2`dataframe, without low counts genes.

**Factorisation**  

We have to tell DESeq2 which information is used as a *factor*, meaning the one that is going to be used as a way to separate groups for future comparison.   
This is done by using the `factor()` function, and providing the name of the column containing this information, and the various values that this info can take.   
Feeding this factorisation into our samples dataframe changes the way this column will be read: as a factor istead of mere characters. 
   
   The `relevel()` function can also be used to clearly specify which level is used as a reference. Most of the times, it will be the WT condition.  


In [ ]:
## Code cell 14 ##

samples$Condition = factor(samples$Condition, levels = c("WT", "dHet", "dHetRag" ))

In [ ]:
## Code cell 15 ##

samples$Condition = relevel(samples$Condition, ref = "WT")

We can verify that this factorisation worked, by showing how many samples were placed in each group:

In [ ]:
## Code cell 16 ##

table(samples$Condition)

and by checking the levels included in that factor:

In [ ]:
## Code cell 17 ##

levels(samples$Condition)

If we look at our `samples`dataframe, we can now see that our `Condition`column is marked as a factor: **fct**

In [ ]:
## Code cell 18 ##

head(samples, n = 2)

We can now create the `dds` object from our two dataframes:

In [ ]:
## Code cell 19 ##

# round() is used as a security, because DESeq2 works only in integers. Read counts should be integers.

dds <- DESeq2::DESeqDataSetFromMatrix(countData = round(countdata2),
                              colData = samples,
                              design = ~Condition)

We check the number of genes in the dataset

In [ ]:
## Code cell 20 ##

nrow(dds)

# 57010 in countdata
# 22927 in countdata2

## 2.2 - Run the DESeq function to analyse the dds object

Next, the DEseq function performs the following steps on the dds object we created:

    Estimation of size factors
    Estimation of dispersion
    Negative Binomial GLM fitting and Wald statistic 
    
For detailed explanation, please refer to the DESeq2 paper.

In [ ]:
## Code cell 21 ##

dds2 <- DESeq2::DESeq(dds)

Let's inspect the header of the newly created object

In [ ]:
## Code cell 22 ##

head(dds2)

The portion `assay` within `dds` contains the expression data, still not normalized :

In [ ]:
## Code cell 23 ##

head(assay(dds2))

In [ ]:
## Code cell 24 ##

summary(assay(dds2))

We can verify this by plotting the distribution of read counts (with a log2 transformation) using a boxplot, to verify that the read counts have indeed not yet been normalized between samples.

In [ ]:
## Code cell 25 ##

# Get log2 counts
logcountsdds2 <- log2(assay(dds2) + 1)
# summary(logcounts[,1]) # summary for first column
 summary(logcountsdds2) # summary for each column

In [ ]:
## Code cell 26 ##

# make a colour vector
conditionColor <- match(samples$Condition, c("dHet", "dHetRag", "WT")) + 1
# '+1' to avoid color '1' i.e. black

# Check distributions of samples using boxplots
boxplot(logcountsdds2,
        xlab="",
        ylab="Log2(dds2 Counts)",
        las=2,
        col=conditionColor,
        main="Log2(dds2 Counts)")
# Let's add a blue horizontal line that corresponds to the median
abline(h=median.default(as.matrix(logcountsdds2)), col="blue")


As we can see, the log2 transformation gives a data distribution closer to the normal distribution, but does not perform well for normalizing data between samples.    

The main motive behind normalization is to change numeric values in a dataset to a common scale without distorting the differences between samples. This reduces the sparsity of the data as they are brought to a common close scale. There are several normalization methods that can be applied to our data.

DESeq2 provides 2 normalisation methods: `rlog`and `vst`.  
We will test the effect of both. 

## 2.3 - Data normalisation with `rlog`: Regularized-logarithm transformation

This function transforms the count data to the log2 scale in a way which minimizes differences between samples for rows with small counts, and which normalizes with respect to library size.

> *In the function calls, we specified `blind = FALSE`, which means that differences between cell lines and treatment (the variables in the design) will not contribute to the expected variance-mean trend of the experiment. The experimental design is not used directly in the transformation, only in estimating the global amount of variability in the counts. For a fully **unsupervised** transformation, one can set `blind = TRUE` (which is the default).*   
[Love MI *et al.*, 2019](https://www.bioconductor.org/packages/release/workflows/vignettes/rnaseqGene/inst/doc/rnaseqGene.html#the-variance-stabilizing-transformation-and-the-rlog)

`blind=FALSE` greatly reduces the run time.

In [ ]:
## Code cell 27 ##

rlog.dds2 <- rlog(dds2, blind = FALSE)
rlog.dds2

Let's verify the effect by plotting the counts from `rlog.dds2`

In [ ]:
## Code cell 28 ##

# Get log2 counts
counts.rlog.dds2 <- assay(rlog.dds2)

summary(counts.rlog.dds2) # summary for each column

In [ ]:
## Code cell 29 ##

# make a colour vector
conditionCol <- match(samples$Condition, c("dHet", "dHetRag", "WT")) + 1
# '+1' to avoid color '1' i.e. black

# Check distributions of samples using boxplots
boxplot(counts.rlog.dds2,
        xlab="",
        ylab="rlog.dds2 Counts",
        las=2,
        col=conditionCol,
        main="rlog.dds2 Counts")
# Let's add a blue horizontal line that corresponds to the median
abline(h=median.default(counts.rlog.dds2), col="blue")

We can verify that variance is not correlated any more to the average level of expression by plotting standard deviation vs mean expression:

In [ ]:
## Code cell 30 ##

# Raw counts mean expression Vs standard Deviation (SD)
plot(rowMeans(assay(rlog.dds2)), matrixStats::rowSds(as.matrix(assay(rlog.dds2)), na.rm=TRUE), 
     main='rlog normalized read counts: sd vs mean', 
     xlim=c(-1,17),
     ylim=c(-1,5))

## 2.4 - Normalization with `vst`: variance stabilizing transformation

Variance stabilizing transformation (VST) aims at generating a matrix of values for which variance is constant across the range of mean values, especially for low mean.

The vst function computes the fitted dispersion-mean relation, derives the transformation to apply and accounts for library size.

In [ ]:
## Code cell 31 ##

vst.dds2 = vst(dds2, blind=FALSE)
vst.dds2

Again, we check the effect by plotting the log2 counts from `vst.dds2`

In [ ]:
## Code cell 32 ##

# Get log2 counts
counts.vst.dds2 <- assay(vst.dds2)

summary(counts.vst.dds2) # summary for each column

In [ ]:
## Code cell 33 ##

# make a colour vector
conditionCol <- match(samples$Condition, c("dHet", "dHetRag", "WT")) + 1
# '+1' to avoid color '1' i.e. black

# Check distributions of samples using boxplots
boxplot(counts.vst.dds2,
        xlab="",
        ylab="vst.dds2 Counts",
        las=2,
        col=conditionCol,
        main="vst.dds2 Counts")
# Let's add a blue horizontal line that corresponds to the median
abline(h=median.default(counts.vst.dds2), col="blue")

We also verify the decoupling of variance and mean level of expression:

In [ ]:
## Code cell 34 ##

# Raw counts mean expression Vs standard Deviation (SD)
plot(rowMeans(assay(vst.dds2)), matrixStats::rowSds(as.matrix(assay(vst.dds2)), na.rm=TRUE), 
     main='vst normalized read counts: sd vs mean', 
     xlim=c(5,17),
     ylim=c(-1,5))

## 2.5- Choosing best transformation for our data

You can see that both methods worked on our data, even if the output are different.   
    
The indications to guide the choice between the two methods are:

> *For genes with high counts, both the VST and the rlog will give similar result to the ordinary log2 transformation of normalized counts. For genes with lower counts, however, the values are shrunken towards a middle value. The VST or rlog-transformed data then become approximately homoskedastic (more flat trend in the meanSdPlot), and can be used directly for computing distances between samples, making PCA plots, or as input to downstream methods which perform best with homoskedastic data.  
> **Which transformation to choose?** The VST is much faster to compute and is less sensitive to high count outliers than the rlog. The rlog tends to work well on small datasets (n < 30), potentially outperforming the VST when there is a wide range of sequencing depth across samples (an order of magnitude difference). We therefore recommend the VST for medium-to-large datasets (n > 30). You can perform both transformations and compare the meanSdPlot or PCA plots generated, as described below.*  
[Love MI *et al.*, 2019](https://www.bioconductor.org/packages/release/workflows/vignettes/rnaseqGene/inst/doc/rnaseqGene.html#the-variance-stabilizing-transformation-and-the-rlog)


Both tranformation methods performed well on our dataset, but we can notice that, in the rlog normalisation, the distribution appear more centered on the median for each sample, and the variance seems less related to the mean level of expression, especially for low values. Furthermore, there are still some outliers for the vst nomalisation.  

Thus, we will keep the **rlog** transformation to go on with the the second exploratory analysis in the next notebook. 

---
## 3 - Differential expression analysis with `DESeq2`

### 3.1 - Building the results table

The `results` table is a dataframe that contains our differentially expressed genes, their p-values, whether the genes are upregulated or downregulated, the number of outliers, etc.  
`summary()` gives us the genes that are up and down regulated in our condition under study, as well as low count genes and outliers. This generates summary tarries for the data. You can read about more about the results function by looking up ?results

We begin by defining the contrast, that is the comparison that we want to perform.    
Here we choose to compare the ALL cells to the WT ones: 

In [ ]:
## Code cell 35 ##

contrastALL=c("Condition", "dHet","WT")

Now we can run DESeq2 for this comparison. It will not work on the normalised rlog.dds2, but on the not-yet normalised dds2 : indeed the normalisation step has to take into account the inter-group variability, and the intra-group variability to be correct, meaning that... the groups have to be defined first!   
Now that DESeq2 knows which group we want to compare, thanks to the `contrast` defined above, it will run the normalisation on the fly, and identify the genes that are significantly differentially expressed (DE genes) according to that specific normalisation.  

In [ ]:
## Code cell 36 ##

res_table <- results(dds2, contrast = contrastALL)
summary(res_table)

Here, we found very few outliers, and we have 9.7% of low counts genes.   
The threshold used to considered a gene as significantly diffrentially expressed is a adjusted p-value of 0.1. Adjusted p-value means that a correction have been performed on the raw p-value, to take into consideration that the statistical test have been reiterated many times, as many as the number of genes. On such a high number of tests, some will give a "significant result" only by chance.   
The most-commonly used p-value correction is the **Benjamini-Hochberg method**, it is the one used by default by `DESeq2`. You could specify another one in the argument of the function.  

We obtain 3309 significantly up-regulated genes + 3542 significantly down-regulated genes. LFC stands for Log2 Fold Change, that is the ratio of the expression of a given gene in the test conditon on its expression in the Reference condition, in log2 scale. Therefore, un log2 FC of 1 means that the gene is 2-times more expressed in the test samples versus the reference samples.

### 3.2 - Working with a different threshold: alpha 0.05

By default, the result function uses an adjusted p-value cutoff of 0.1. If any other p value is prefered, `alpha` should be set to that value, as shown below where we set `alpha=0.05`.

In [ ]:
## Code cell 37 ##

results2 <- results(dds2, alpha=0.05)

We can now count the number of differentially expressed genes with adjusted p-values were less than 0.05. na.rm=TRUE removes any row with missing information.

In [ ]:
## Code cell 38 ##

sum(results2$padj < 0.05, na.rm=TRUE)
summary(results2)

# 2759 

So out of the 22927 filtered genes, only 2759 have adjusted p-values less than 0.05, with 909 significantly up-regulated genes + 1850 significantly down-regulated genes. Let’s go ahead and select them.

In [ ]:
## Code cell 39 ##

res2_sig_genes <- subset(results2, padj < 0.05)
dim(res2_sig_genes)

## 2759     6

We then order the selected genes by the smallest p value:

In [ ]:
## Code cell 40 ##

res2_ranked_p <- res2_sig_genes[order(res2_sig_genes$pvalue),]
dim(res2_ranked_p)

## [1] 2759     6

Finally, we write the differentially expressed gene list to a tabulated txt file, in the deseq2folder that we decided at first:

In [ ]:
## Code cell 41 ##

write.table(as.data.frame(res2_ranked_p), file=paste0(deseq2folder,"DESeq2_significant_genes-0_05.tsv"), sep="\t", quote=F, col.names=T)

## 4 - Saving our results for later use: RData file

We can save all the R objects created in this session in a single R object.   
This will help us to reload our results without having to run the same commands.   

In [ ]:
## Code cell 42 ##

print(ls())

We keep only the relevant objects:

In [ ]:
## Code cell 43 ##

rm(conditionCol, countdata, dds, res_table, results2, res2_sig_genes, vst.dds2)

and we save all our info in a single RData object in our output folder:

In [ ]:
## Code cell 44 ##

ls()
save.image(file=paste0(deseq2folder,"deseq2.RData"))

---
___

Now we go on with the visualization of normalized read counts in a full exploratory analysis.  
  
**=> Step 10: Exploratory analysis of normalized data** 

The jupyter notebook used for the next session will be *Pipe_10-R403-Normcounts-exploratory-analysis-II.ipynb*    
Let's retrieve it in our directory, in order to have a private copy to work on:   

In [ ]:
## Code cell 45 ##   

myfolder

file.copy("/shared/projects/2312_rnaseq_cea/pipeline/Pipe_10-R403-Normcounts-exploratory-analysis-II.ipynb", myfolder)




**Save executed notebook**

To end the session, save your executed notebook in your `run_notebooks' folder. **Adjust the name with yours** and reformat as code cell to run it.

In [ ]:
## Code cell 46 ##   

# creation of the directory, recursive = TRUE is equivalent to the mkdir -p in Unix
dir.create(paste0(myfolder,"/run_notebooks"), recursive = TRUE)

runfolder <- paste0(myfolder,"/run_notebooks")
       
file.copy(paste0(myfolder, "/Pipe_09-R403-DESeq2-normalisation-DE.ipynb"), runfolder)


<div class="alert alert-block alert-success"><b>Success:</b> Well done! You now know how to perform a normalisation and a differential expression analysis using DESeq2.<br>
Don't forget to save you notebook and export a copy as an <b>html</b> file as well <br>
- Open "File" in the Menu<br>
- Select "Export Notebook As"<br>
- Export notebook as HTML<br>
- You can then open it in your browser even without being connected to the server! 
</div>

---
---

## Useful commands
<div class="alert alert-block alert-info"> 
    
- <kbd>CTRL</kbd>+<kbd>S</kbd> : save notebook<br>    
- <kbd>CTRL</kbd>+<kbd>ENTER</kbd> : Run Cell<br>  
- <kbd>SHIFT</kbd>+<kbd>ENTER</kbd> : Run Cell and Select Next<br>   
- <kbd>ALT</kbd>+<kbd>ENTER</kbd> : Run Cell and Insert Below<br>   
- <kbd>ESC</kbd>+<kbd>y</kbd> : Change to *Code* Cell Type<br>  
- <kbd>ESC</kbd>+<kbd>m</kbd> : Change to *Markdown* Cell Type<br> 
- <kbd>ESC</kbd>+<kbd>r</kbd> : Change to *Raw* Cell Type<br>    
- <kbd>ESC</kbd>+<kbd>a</kbd> : Create Cell Above<br> 
- <kbd>ESC</kbd>+<kbd>b</kbd> : Create Cell Below<br> 

<em>  
To make nice html reports with markdown: <a href="https://dillinger.io/" title="dillinger.io">html visualization tool 1</a> or <a href="https://stackedit.io/app#" title="stackedit.io">html visualization tool 2</a>, <a href="https://www.tablesgenerator.com/markdown_tables" title="tablesgenerator.com">to draw nice tables</a>, and the <a href="https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd" title="Ultimate guide">Ultimate guide</a>. <br>
Further reading on JupyterLab notebooks: <a href="https://jupyterlab.readthedocs.io/en/latest/user/notebook.html" title="Jupyter Lab">Jupyter Lab documentation</a>.<br>   
</em>    
 
</div>

Bénédicte Noblet - 05-07 2021   
Sandrine Caburet et Claire Vandiedonck - 02-06 2023   
with adaptations from https://bioinformatics-core-shared-training.github.io/RNAseq_November_2020_remote/html/02_Preprocessing_Data.html   
and https://rpubs.com/adoughan/778146  
Maj 01/06/2023 par @SCaburet   